In [242]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeCV
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import mean_squared_error
from scipy.stats.mstats import normaltest
import patsy
import numpy as np
import random

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [243]:
# get last year's ESPN top 750 leading fantasy point scorers

leader_names = []
leader_points = []

count = 0
while count < 15:
    url1 = 'http://games.espn.com/ffl/leaders?&startIndex='+str(count*50)
    print(url1)
    response1 = requests.get(url1)
    print(response1)
    page1 = response1.text
    soup1 = BeautifulSoup(page1, "lxml")
    
    for thing in soup1.find_all('td'):
        for thing2 in thing.find_all(class_='playertablePlayerName'):
            leader_names.append(thing2.find('a').text)

    for thing in soup1.find_all('td'):
        for thing2 in thing.find_all(class_='playertableStat appliedPoints sortedCell'):
            leader_points.append(float(thing2.text))
            
    count = count + 1

#print(soup1.prettify())



http://games.espn.com/ffl/leaders?&startIndex=0
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=50
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=100
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=150
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=200
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=250
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=300
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=350
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=400
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=450
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=500
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=550
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=600
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=650
<Response [200]>
http://games.espn.com/ffl/leaders?&startIndex=700
<Response [200]

In [244]:
# get last year's ESPN projections for the top ranked 300 fantasy players
url2 = 'http://www.espn.com/fantasy/football/story/_/id/16287927/2016-fantasy-football-rankings-top-300'
response2 = requests.get(url2)
page2 = response2.text
soup2 = BeautifulSoup(page2, "lxml")

In [245]:
#make lists of player names, their positions, and their pre-season ranking 
projection_names = []
projection_rank = []
positions = []
for i, thing in enumerate(soup2.find_all(class_ = 'last')):
    if i not in [0,1,2]: #the first several elements are not players
        entry = thing.find('td').text.split()
        projection_rank.append(int(entry[0][:-1])) #save rank and get rid of period at end
        projection_names.append(' '.join(entry[1:]))
        positions.append(list(thing.find_all('td'))[1].text)
positions[0:10]

['WR', 'WR', 'RB', 'WR', 'RB', 'RB', 'WR', 'RB', 'RB', 'WR']

In [113]:
for i,name in enumerate(leader_names):
    print(i,name)

0 David Johnson
1 Aaron Rodgers
2 Matt Ryan
3 Drew Brees
4 Ezekiel Elliott
5 Le'Veon Bell
6 Andrew Luck
7 Antonio Brown
8 Jordy Nelson
9 Mike Evans
10 Kirk Cousins
11 LeSean McCoy
12 Odell Beckham Jr.
13 DeMarco Murray
14 Dak Prescott
15 Devonta Freeman
16 Matthew Stafford
17 T.Y. Hilton
18 Blake Bortles
19 Tyrod Taylor
20 Russell Wilson
21 Derek Carr
22 Andy Dalton
23 Marcus Mariota
24 Julio Jones
25 Philip Rivers
26 Tom Brady
27 Jameis Winston
28 Michael Thomas
29 Cam Newton
30 Doug Baldwin
31 Ben Roethlisberger
32 Melvin Gordon
33 Davante Adams
34 Brandin Cooks
35 Larry Fitzgerald
36 Carson Palmer
37 Joe Flacco
38 Mark Ingram
39 Michael Crabtree
40 LeGarrette Blount
41 Amari Cooper
42 Jarvis Landry
43 Julian Edelman
44 Jordan Howard
45 Demaryius Thomas
46 Alex Smith
47 Eli Manning
48 Golden Tate
49 Travis Kelce
50 Sam Bradford
51 Tyreek Hill
52 Tyrell Williams
53 Jay Ajayi
54 Frank Gore
55 Carson Wentz
56 Terrelle Pryor Sr.
57 Emmanuel Sanders
58 Rishard Matthews
59 Kyle Rudolph
60 

675 Seyi Ajirotutu
676 Richie Brockel
677 Jeff Cumberland
678 Thaddeus Lewis
679 Marlon Moore
680 Danario Alexander
681 Tyler Clutts
682 Christian Ponder
683 Rob Housler
684 Leonard Hankerson
685 Greg Little
686 Vincent Brown
687 Ryan Williams
688 Daniel Thomas
689 Bruce Miller
690 David Ausberry
691 T.J. Yates
692 Denarius Moore
693 D.J. Williams
694 Kendall Hunter
695 Roy Helu Jr.
696 Jerrod Johnson
697 Ricardo Lockette
698 Joe Morgan
699 Will Johnson
700 Brandon Weeden
701 Kellen Moore
702 Ryan Lindley
703 Trent Richardson
704 Chris Polk
705 LaMike James
706 Bernard Pierce
707 Cyrus Gray
708 Vick Ballard
709 Orson Charles
710 James Hanna
711 Justin Blackmon
712 Stephen Hill
713 Rueben Randle
714 T.J. Graham
715 Nick Toon
716 DeVier Posey
717 LaVon Brazill
718 Daniel Herron
719 Chris Givens
720 Bryce Brown
721 Greg Scruggs
722 Kashif Moore
723 Austin Davis
724 Nathan Palmer
725 Cory Harkey
726 Chris Owusu
727 Brian Tyms
728 Jeff Fuller
729 Derrick Coleman
730 Jarrett Boykin
731 Jonas

In [246]:
#make a dictionary mapping each player to their position, their projected ranking, and their actual earned points

projection_dict={}
for player in projection_names:
    projection_dict[player] = [positions[projection_names.index(player)],projection_rank[projection_names.index(player)]]

leader_dict={}
for player in leader_names:
    leader_dict[player] = leader_points[leader_names.index(player)]
leader_dict

player_dict = {}
for player in projection_dict:
    if player in leader_dict: #most names are the same in both lists
        player_dict[player] = projection_dict[player]
        player_dict[player].append(leader_dict[player])

    elif player in str(leader_dict): # a few names are different because of suffixes
        for key2 in leader_dict.keys():
            if player in key2:
                player_dict[player] = projection_dict[player]
                player_dict[player].append(leader_dict[key2])
                
#    elif player.split(' ')[1] in str(leader_dict): #one-word cities need to be deleted from defense names        
#        if player != 'Maxx Williams': #got 0 points so not in top 750 scorers and name is problematic
#            player_dict[player] = projection_dict[player]
#            player_dict[player].append(leader_dict[player.split(' ')[1]+' D/ST'])
            
#    elif player.split(' ')[2] in str(leader_dict): #two-word cities need to be deleted from defense names     
#        player_dict[player] = projection_dict[player]
#        player_dict[player].append(leader_dict[player.split(' ')[2]+' D/ST'])
#    else:
#        print(player, 'not in both')
        
        
    elif player.split(' ')[1] in str(leader_dict): #one-word cities need to be deleted from defense names    
        print(player)
        if player != 'Maxx Williams': #got 0 points so not in top 750 scorers
            player_dict[player] = projection_dict[player]
            player_dict[player].append(leader_dict[player.split(' ')[1]+' D/ST'])
            
    elif player.split(' ')[2] in str(leader_dict): #two-word cities need to be deleted from defense names     
        player_dict[player] = projection_dict[player]
        player_dict[player].append(leader_dict[player.split(' ')[2]+' D/ST'])
    else:
        print(player, 'not in both')
#edge cases with different names in later data sets
player_dict['Rob Kelley'] = player_dict['Robert Kelley']
del player_dict['Robert Kelley']

player_dict['Benny Cunningham'] = player_dict['Benjamin Cunningham']
del player_dict['Benjamin Cunningham']

player_dict

Seattle Seahawks
Denver Broncos
Carolina Panthers
Cincinnati Bengals
Houston Texans
Arizona Cardinals
Oakland Raiders
Minnesota Vikings
Buffalo Bills
Indianapolis Colts
Jacksonville Jaguars
Baltimore Ravens
Pittsburgh Steelers
Maxx Williams
Philadelphia Eagles
Chicago Bears
Detroit Lions


{'A.J. Green': ['WR', 10, 186.4],
 'Aaron Rodgers': ['QB', 49, 380.0],
 'Adam Humphries': ['WR', 262, 133.0],
 'Adam Thielen': ['WR', 295, 195.2],
 'Adam Vinatieri': ['K', 155, 145.0],
 'Adrian Peterson': ['RB', 3, 9.0],
 'Albert Wilson': ['WR', 272, 81.9],
 'Alex Smith': ['QB', 261, 224.8],
 'Alfred Blue': ['RB', 277, 62.0],
 'Alfred Morris': ['RB', 161, 40.4],
 'Allen Hurns': ['WR', 80, 100.7],
 'Allen Robinson': ['WR', 13, 199.3],
 'Alshon Jeffery': ['WR', 21, 146.1],
 'Amari Cooper': ['WR', 29, 232.3],
 'Ameer Abdullah': ['RB', 75, 26.8],
 'Andre Ellington': ['RB', 247, 30.1],
 'Andre Johnson': ['WR', 274, 29.5],
 'Andrew Luck': ['QB', 61, 307.7],
 'Andy Dalton': ['QB', 170, 260.6],
 'Anquan Boldin': ['WR', 183, 173.4],
 'Antonio Andrews': ['RB', 280, 1.5],
 'Antonio Brown': ['WR', 1, 307.3],
 'Antonio Gates': ['TE', 111, 147.8],
 'Arian Foster': ['RB', 74, 19.3],
 'Arizona Cardinals': ['DST', 147, 143.0],
 'Austin Seferian-Jenkins': ['TE', 210, 34.4],
 'Baltimore Ravens': ['DST', 

In [247]:
#drop defenses
df = df [ df['position'] != 'DST']
df.shape

(277, 3)

In [248]:
#make this dictionary into a data frame
df = pd.DataFrame(player_dict).T
df = df.rename(index=str, columns={0: "position", 1: "projection", 2: "points"})
df.sort_values(by='points',ascending= False).head()

,position,projection,points
David Johnson,RB,8,407.8
Aaron Rodgers,QB,49,380
Matt Ryan,QB,195,347.5
Drew Brees,QB,68,332.3
Ezekiel Elliott,RB,6,325.4


In [11]:
##DON'T RUN AGAIN!!! THIS GETS ALL 168 PAGES OF ALL PLAYERS' INFO

#url3 = 'http://www.foxsports.com/nfl/players?teamId=0&season=2016&position=0&page=' + str(1) + '&country=0&grouping=0&weightclass=0'
#players = pd.read_html(url3)[0]

#for i in range(1,168):
#    url3 = 'http://www.foxsports.com/nfl/players?teamId=0&season=2016&position=0&page=' + str(i+1) + '&country=0&grouping=0&weightclass=0'
#    table2 = pd.read_html(url3)[0]
#    players = pd.concat([players,table2], ignore_index = True)

#CLEANS UP PLAYERS' NAMES
#players.columns = [column.strip() for column in players.columns]
#for i in range(len(players)):
#    players.set_value(i, 'Player', players['Player'][i].split(',')[1].strip())
#players.Player = [man.replace('  ', ' ') for man in players.Player]

#SAVE FILE
#players.to_pickle('allplayers.pkl')

In [249]:
#get player stat info from fox:
allplayers = pd.read_pickle('allplayers.pkl')

In [120]:
allplayers.head()

,Player,Team,No,Pos,Exp,Height,Weight,Age,School
0,Isaako Aaitui,TEN,71,DT,2,"6'4""",307,29,UNLV
1,Jared Abbrederis,DET,-,WR,3,"6'1""",195,25,Wisconsin
2,Mehdi Abdesmad,TEN,92,DE,0,"6'6""",284,24,Boston College
3,Isa Abdul-Quddus,MIA,24,S,6,"6'1""",196,26,Fordham
4,Ameer Abdullah,DET,21,RB,2,"5'9""",203,23,Nebraska


In [250]:
### merge both data sets

#set player name equal to index
allplayers.set_index(['Player'], inplace = True)

#first match up player names such as suffixes
for i, name in enumerate(df.index.values):
    if name in allplayers.index.values:
        #print(i, name)
        continue
    else:
        maxfz = 0
        closest_name = ''
        for i, player_name in enumerate(allplayers.index.values):  #takes care of suffixes
            fz = fuzz.partial_ratio(name, player_name)
            if fz > maxfz:
                maxfz = fz
                closest_name = player_name
        if maxfz == 100:
            print(name, closest_name, maxfz)
            df.rename(index={name: closest_name}, inplace = True)
            
#then merge            
del df.index.name
del allplayers.index.name
results = pd.merge(df, allplayers, how='inner', left_index=True, right_index = True)

Marvin Jones Marvin Jones Jr. 100
Robert Griffin Robert Griffin III 100
Terrelle Pryor Terrelle Pryor Sr. 100
Will Fuller Will Fuller V 100


In [141]:
#We picked up a few duplicates because there were some players with the same name in the full list of NFL players or have errors
results.loc['Alex Smith']

,position,projection,points,Team,No,Pos,Exp,Height,Weight,Age,School
Alex Smith,QB,261,224.8,KC,11,QB,12,"6'4""",217,32,Utah
Alex Smith,QB,261,224.8,WSH,83,TE,12,"6'4""",250,34,Stanford


In [228]:
results.shape

(283, 11)

In [123]:
#Find the duplicates:
for name in results.index:
    if list(results.index).count(name) > 1:
        print(name)

Alex Smith
Alex Smith
Brandon Marshall
Brandon Marshall
Charles Johnson
Charles Johnson
David Johnson
David Johnson
Kevin White
Kevin White
Michael Thomas
Michael Thomas


In [251]:
#keep only the right entries
results = results[(results.index != 'Brandon Marshall') | (results.School != 'Nevada') ]
results = results[(results.index != 'Michael Thomas') | (results.School != 'Stanford') ]
results = results[(results.index != 'Kevin White') | (results.School != 'TCU') ]
results = results[(results.index != 'David Johnson') | (results.School != 'Arkansas State') ]
results = results[(results.index != 'Alex Smith') | (results.School != 'Stanford') ]
results = results[(results.index != 'Charles Johnson') | (results.School != 'Georgia') ]
results.shape

(277, 11)

In [252]:
# Rank universities by most championships
url4 = 'https://en.wikipedia.org/wiki/College_football_national_championships_in_NCAA_Division_I_FBS'
univ = pd.read_html(url4)[9]

univ = univ.iloc[1:, 0:2]
univ = univ.rename(index=str, columns={0: "School", 1: "School_Championships"})
univ.head()

,School,School_Championships
1,Alabama,11
2,Notre Dame,8
3,Oklahoma,7
4,USC,7
5,Ohio State,6


In [254]:
## Merge data sets

results_with_univ = (
    pd.merge(results.reset_index(), univ, how='left', left_on = 'School', right_on = 'School')
    .set_index('index', inplace=False)
)

results_with_univ.fillna(0, inplace = True)

results_with_univ.head()

,position,projection,points,Team,No,Pos,Exp,Height,Weight,Age,School,School_Championships
index,,,,,,,,,,,,
A.J. Green,WR,10,186.4,CIN,18,WR,6,"6'4""",210,27,Georgia,1
Aaron Rodgers,QB,49,380.0,GB,12,QB,12,"6'2""",225,32,California,0
Adam Humphries,WR,262,133.0,TB,11,WR,2,"5'11""",195,23,Clemson,2
Adam Thielen,WR,295,195.2,MIN,19,WR,3,"6'2""",200,25,Minnesota State-Mankato,0
Adam Vinatieri,K,155,145.0,IND,4,K,21,"6'0""",206,43,South Dakota State,0


In [255]:
# Count how many players in this list came from each school and make a column out of that
popular_schools = pd.DataFrame(results_with_univ.School.value_counts())
popular_schools = popular_schools.rename(index=str, columns={"School": "Num_of_players_from_school"})
popular_schools.reset_index(inplace = True)
popular_schools = popular_schools.rename(index=str, columns={"index": "School"})
popular_schools.head()

,School,Num_of_players_from_school
0,Miami (FL),10
1,California,7
2,Alabama,7
3,Ohio State,7
4,USC,7


In [256]:
# Merge Data Sets
results_with_univ_counts = (
    pd.merge(results_with_univ.reset_index(), popular_schools, how='left', left_on = 'School', right_on = 'School')
    .set_index('index', inplace=False)
)
results_with_univ_counts.head()

,position,projection,points,Team,No,Pos,Exp,Height,Weight,Age,School,School_Championships,Num_of_players_from_school
index,,,,,,,,,,,,,
A.J. Green,WR,10,186.4,CIN,18,WR,6,"6'4""",210,27,Georgia,1,6
Aaron Rodgers,QB,49,380.0,GB,12,QB,12,"6'2""",225,32,California,0,7
Adam Humphries,WR,262,133.0,TB,11,WR,2,"5'11""",195,23,Clemson,2,7
Adam Thielen,WR,295,195.2,MIN,19,WR,3,"6'2""",200,25,Minnesota State-Mankato,0,1
Adam Vinatieri,K,155,145.0,IND,4,K,21,"6'0""",206,43,South Dakota State,0,2


In [193]:
results_with_univ_counts.shape

(277, 13)

In [257]:
#import depth chart from https://www.reddit.com/r/nfl/comments/4n2uzj/2016_nfl_depth_charts_all_teams_all_positions/
depth = pd.read_csv('depth.csv')
depth['Player'] = depth.Player.str.title()
depth.head()

,Player,Position,Depth
0,Sammy Watkins,LWR,1
1,Marquise Goodwin,LWR,2
2,Kolby Listenbee,LWR,3
3,Dezmin Lewis,LWR,4
4,Greg Little,LWR,5


In [258]:
# Resolve discrepancy in names between depth chart and our data. Print the ones that are still problems
for name in results_with_univ_counts.index:
    problem_solved = False
    if name not in depth.Player.values:
        maxfz = 0
        closest_name = ''
        for i, player_name in enumerate(depth.Player.values):  #takes care of suffixes
            fz = fuzz.partial_ratio(name, player_name)
            if fz > maxfz:
                maxfz = fz
                closest_name = player_name
                saved_index = i
            if name.replace('.', '').lower() == player_name.replace('.', '').lower(): #initials
                depth = depth.set_value(i, 'Player', name)
#                print(name, closest_name)
                problem_solved = True
                continue
            if name.replace('.', '').lower() == player_name.replace(',', ' ').lower(): #weird spacing for steve smith
                depth = depth.set_value(i, 'Player', name)
#                print(name, closest_name)
                problem_solved = True
                continue
            if '-' in name:
                if name.split(' ')[0]+ ' ' +name.split(' ')[1].split('-')[1]  == player_name: #hyphens
                    depth = depth.set_value(i, 'Player', name)
#                    print(name, closest_name)
                    problem_solved = True
                    continue
        if maxfz == 100: #suffixes
            depth = depth.set_value(saved_index, 'Player', name)
            problem_solved = True
#            print(name, closest_name, maxfz)
        if problem_solved == False:
            print(name, closest_name, maxfz)

Andre Johnson Randell Johnson 85
Anquan Boldin Brandon Bolden 64
Arian Foster Dj Foster 78
Reggie Bush Reggie Nelson 73
Ryan Fitzpatrick Ryan Smith 70


In [260]:
# get rid of these five names that don't have depth charts attached to them
getrid = []
for name in results_with_univ_counts.index:
    if name not in depth.Player.values:
        getrid.append(name) 
print(getrid)
results_with_univ_counts = results_with_univ_counts.loc[results_with_univ_counts.index.drop(getrid)]
results_with_univ_counts.shape

[]


(272, 13)

In [261]:
# merge data sets
depth = depth.set_index(['Player'])
depth.head()

,Position,Depth
Player,,
Sammy Watkins,LWR,1
Marquise Goodwin,LWR,2
Kolby Listenbee,LWR,3
Dezmin Lewis,LWR,4
Greg Little,LWR,5


In [262]:
#many names have more than one position listed in depth (Such as punt returner and wide receiver). Keep only the main job
depth = depth[depth.Position != 'PR']
depth = depth[depth.Position != 'KR']
depth = depth[depth.Position != 'LS']
depth = depth[depth.Position != 'KO']
depth = depth[depth.Position != 'LDE']
depth = depth[depth.Position != 'FB']
depth = depth[depth.Position != 'ILB']
depth = depth[depth.Position != 'P']
depth = depth[depth.Position != 'FS']
depth = depth[depth.Position != 'H']

In [147]:
# confirm that there are now unique names with positions
for i,name in enumerate(results_with_univ_counts.index):
    print(i, name, depth.loc[name].Position, results_with_univ_counts.loc[name].position)

0 A.J. Green LWR WR
1 Aaron Rodgers QB QB
2 Adam Humphries RWR WR
3 Adam Thielen LWR WR
4 Adam Vinatieri PK K
5 Adrian Peterson RB RB
6 Albert Wilson LWR WR
7 Alex Smith QB QB
8 Alfred Blue RB RB
9 Alfred Morris RB RB
10 Allen Hurns RWR WR
11 Allen Robinson LWR WR
12 Alshon Jeffery RWR WR
13 Amari Cooper RWR WR
14 Ameer Abdullah RB RB
15 Andre Ellington RB RB
16 Andrew Luck QB QB
17 Andy Dalton QB QB
18 Antonio Andrews RB RB
19 Antonio Brown LWR WR
20 Antonio Gates TE TE
21 Austin Seferian-Jenkins TE TE
22 Ben Roethlisberger QB QB
23 Benny Cunningham RB RB
24 Bilal Powell RB RB
25 Blaine Gabbert QB QB
26 Blair Walsh PK K
27 Blake Bortles QB QB
28 Brandin Cooks RWR WR
29 Brandon Bolden RB RB
30 Brandon LaFell RWR WR
31 Brandon Marshall LWR WR
32 Brandon McManus PK K
33 Braxton Miller LWR WR
34 Breshad Perriman LWR WR
35 Brock Osweiler QB QB
36 C.J. Anderson RB RB
37 C.J. Prosise RB RB
38 C.J. Spiller RB RB
39 Cairo Santos PK K
40 Cam Newton QB QB
41 Cameron Artis-Payne RB RB
42 Cameron 

In [263]:
#merge data sets

final_results = pd.merge(results_with_univ_counts, depth, how='left', left_index=True, right_index = True)
final_results.head()

,position,projection,points,Team,No,Pos,Exp,Height,Weight,Age,School,School_Championships,Num_of_players_from_school,Position,Depth
A.J. Green,WR,10,186.4,CIN,18,WR,6,"6'4""",210,27,Georgia,1,6,LWR,1
Aaron Rodgers,QB,49,380.0,GB,12,QB,12,"6'2""",225,32,California,0,7,QB,1
Adam Humphries,WR,262,133.0,TB,11,WR,2,"5'11""",195,23,Clemson,2,7,RWR,2
Adam Thielen,WR,295,195.2,MIN,19,WR,3,"6'2""",200,25,Minnesota State-Mankato,0,1,LWR,3
Adam Vinatieri,K,155,145.0,IND,4,K,21,"6'0""",206,43,South Dakota State,0,2,PK,1


In [264]:
#drop extra position columns
del final_results['Pos']
del final_results['Position']

In [265]:
#convert columns to correct data type
final_results['projection'] = final_results['projection'].astype(int)
final_results['points'] = final_results['points'].astype(float)
final_results['Age'] = final_results['Age'].astype(int)
final_results['School_Championships'] = final_results['School_Championships'].astype(int)
final_results['Num_of_players_from_school'] = final_results['Num_of_players_from_school'].astype(int)
final_results['Depth'] = final_results['Depth'].astype(int)

In [151]:
final_results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 272 entries, A.J. Green to Zach Zenner
Data columns (total 13 columns):
position                      272 non-null object
projection                    272 non-null int64
points                        272 non-null float64
Team                          272 non-null object
No                            272 non-null object
Exp                           272 non-null int64
Height                        272 non-null object
Weight                        272 non-null int64
Age                           272 non-null int64
School                        272 non-null object
School_Championships          272 non-null int64
Num_of_players_from_school    272 non-null int64
Depth                         272 non-null int64
dtypes: float64(1), int64(7), object(5)
memory usage: 29.8+ KB


In [152]:
#only five names deleted because not in depth chart and Maxx Williams deleted at beginning because problematic name plus 0 points scored that season
count = 1
for name in df.index:
    if name not in final_results.index:
        print(count,name)
        count = count + 1

1 Andre Johnson
2 Anquan Boldin
3 Arian Foster
4 Reggie Bush
5 Ryan Fitzpatrick


In [266]:
#get total fantasy points from 2015
url5 = 'http://www.pro-football-reference.com/years/2015/fantasy.htm'
response5 = requests.get(url5)
page5 = response5.text
soup5 = BeautifulSoup(page5, "lxml")


In [268]:
#get player names
names = []
for thing in soup5.find_all('td'):
    try:
        names.append(thing['csk'])
    except:
        continue

for name in names:
    if name.isdigit() == True:
        names.remove(name)

len(names)

595

In [269]:
#get player fantasy points - unfortunately this list doesnt contain kickers (or this year's rookies)
points2015 = []
for thing in soup5.find_all('td'):
    if 'fantasy_points' in str(thing):
        points2015.append(thing.text)
len(points2015)



595

In [270]:
points_dict={}
for i in range(len(names)):
    name = ' '.join([names[i].split(',')[-1], names[i].split(',')[0]])
    points_dict[name] = [points2015[i]]
    

In [271]:
for name in final_results.index:
    if name not in points_dict:
        maxfz = 0
        closest_name = ''
        for i, player_name in enumerate(points_dict):  #takes care of suffixes
            fz = fuzz.partial_ratio(name.replace('.', '').lower(), player_name.replace('.', '').lower())
            if fz > maxfz:
                maxfz = fz
                closest_name = player_name
        if maxfz == 100:
            points_dict[name] = points_dict.pop(closest_name)


In [272]:
df2 = pd.DataFrame(points_dict).T
df2 = df2.rename(index=str, columns={0: "2015_points"})

In [285]:
final_final_results = pd.merge(final_results, df2, how='left', left_index=True, right_index = True)

In [286]:
final_final_results['2015_points'] = final_final_results['2015_points'].fillna(value=0)

In [287]:
#fill in a few missing values
for name in final_final_results['2015_points'].index:
    if final_final_results.loc[name]['2015_points'] == '':
        final_final_results = final_final_results.set_value(name,'2015_points',0)

In [291]:
final_final_results.head()

,position,projection,points,Team,No,Exp,Height,Weight,Age,School,School_Championships,Num_of_players_from_school,Depth,2015_points
A.J. Green,WR,10,186.4,CIN,18,6,"6'4""",210,27,Georgia,1,6,1,188
Aaron Rodgers,QB,49,380.0,GB,12,12,"6'2""",225,32,California,0,7,1,305
Adam Humphries,WR,262,133.0,TB,11,2,"5'11""",195,23,Clemson,2,7,2,34
Adam Thielen,WR,295,195.2,MIN,19,3,"6'2""",200,25,Minnesota State-Mankato,0,1,3,23
Adam Vinatieri,K,155,145.0,IND,4,21,"6'0""",206,43,South Dakota State,0,2,1,0


In [294]:
final_final_results['2015_points'] = final_final_results['2015_points'].astype(int)

In [295]:
final_final_results.to_pickle('final_final_results.pkl')